# ID2221 Project

### Setup:
- start Kafka broker in Terminal and create topic "weather"

In [22]:
from kafka import KafkaProducer, KafkaConsumer
from json import dumps
import time
import json
import requests
import uuid

In [23]:
#personal API key
API_KEY = 'cb37aa19c0716a84475560486afab8a3'                    

city = 'Stockholm'
lat = 59.3326
lon = 18.0649

In [24]:
request = requests.get(f'https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&appid={API_KEY}')
# request = requests.get(f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}')

In [25]:
json_data = json.loads(request.text)
print(json_data)

{'lat': 59.33, 'lon': 18.06, 'timezone': 'Europe/Stockholm', 'timezone_offset': 7200, 'current': {'dt': 1603024775, 'sunrise': 1602999196, 'sunset': 1603035126, 'temp': 278.79, 'feels_like': 270.14, 'pressure': 1002, 'humidity': 48, 'dew_point': 269.16, 'uvi': 0.8, 'clouds': 75, 'visibility': 10000, 'wind_speed': 8.7, 'wind_deg': 300, 'wind_gust': 14.9, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}]}, 'minutely': [{'dt': 1603024800, 'precipitation': 0}, {'dt': 1603024860, 'precipitation': 0}, {'dt': 1603024920, 'precipitation': 0}, {'dt': 1603024980, 'precipitation': 0}, {'dt': 1603025040, 'precipitation': 0}, {'dt': 1603025100, 'precipitation': 0}, {'dt': 1603025160, 'precipitation': 0}, {'dt': 1603025220, 'precipitation': 0}, {'dt': 1603025280, 'precipitation': 0}, {'dt': 1603025340, 'precipitation': 0}, {'dt': 1603025400, 'precipitation': 0}, {'dt': 1603025460, 'precipitation': 0}, {'dt': 1603025520, 'precipitation': 0}, {'dt': 1603025580, 

In [26]:
#Create Kafka Producer
ip = 'localhost'
producer = KafkaProducer(bootstrap_servers=[ip+':9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [27]:
producer.send("weather",json_data)

# Consumer

In [28]:
def connect_kafka_producer(server):
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[server], api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return _producer

In [29]:
kafka_producer = connect_kafka_producer(ip)

In [30]:
consumer = KafkaConsumer(topic_name,
                         auto_offset_reset='earliest',
                         bootstrap_servers=[ip],
                         api_version=(0, 10, 0),
                         value_deserializer = json.loads,
                         consumer_timeout_ms=1000)
for msg in consumer:
    print(msg) #.key.decode("utf-8"), msg.value)

ConsumerRecord(topic='weather', partition=0, offset=0, timestamp=1603017715877, timestamp_type=0, key=None, value={'lat': 59.33, 'lon': 18.06, 'timezone': 'Europe/Stockholm', 'timezone_offset': 7200, 'current': {'dt': 1603017296, 'sunrise': 1602999196, 'sunset': 1603035126, 'temp': 279.62, 'feels_like': 272.94, 'pressure': 1001, 'humidity': 63, 'dew_point': 273.1, 'uvi': 0.8, 'clouds': 40, 'visibility': 10000, 'wind_speed': 6.7, 'wind_deg': 300, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}]}, 'minutely': [{'dt': 1603017300, 'precipitation': 0}, {'dt': 1603017360, 'precipitation': 0}, {'dt': 1603017420, 'precipitation': 0}, {'dt': 1603017480, 'precipitation': 0}, {'dt': 1603017540, 'precipitation': 0}, {'dt': 1603017600, 'precipitation': 0}, {'dt': 1603017660, 'precipitation': 0}, {'dt': 1603017720, 'precipitation': 0}, {'dt': 1603017780, 'precipitation': 0}, {'dt': 1603017840, 'precipitation': 0}, {'dt': 1603017900, 'precipitation': 0}, {'